# E4. 멋진 작사가 만들기 

### Step 0. 필요한 모듈 가져오기

In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence

### Step 1. 데이터 다운로드

이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면, ~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.

### Step 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you', 'By my side But then I spent so many nights', "Just thinking how you've done me wrong", 'I grew strong', "I learned how to get along And so you're back", 'From outer space', 'I just walked in to find you', 'Here without that look upon your face I should have changed that fucking lock']


### Step 3. 데이터 정제

* preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.
* 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.
* 단어장의 크기는 12,000 이상 으로 설정하세요! 


#### 3.1. 정규표현식을(Regex) 이용한 필터링 및 corpus 생성

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

print(len(corpus))

175986


In [5]:
print(corpus[:10])

['<start> at first i was afraid <end>', '<start> i was petrified <end>', '<start> i kept thinking i could never live without you <end>', '<start> by my side but then i spent so many nights <end>', '<start> just thinking how you ve done me wrong <end>', '<start> i grew strong <end>', '<start> i learned how to get along and so you re back <end>', '<start> from outer space <end>', '<start> i just walked in to find you <end>', '<start> here without that look upon your face i should have changed that fucking lock <end>']


#### 3.2. Corpus 를 Tokenize 시켜서 Tensor로 변경

In [28]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=30000, 
    filters=' ',
    oov_token="<unk>"
    )
    
    # tokenizer 내부의 단어장 생성
    tokenizer.fit_on_texts(corpus)
    
    # corpus to Tensor
    tensor = tokenizer.texts_to_sequences(corpus)   

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[   2   71  241 ...    0    0    0]
 [   2    5   57 ...    0    0    0]
 [   2    5 1087 ...    0    0    0]
 ...
 [   2   48   16 ...    0    0    0]
 [  25    9 2859 ...  264   19    3]
 [   2    6  181 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f1255d3add0>


In [29]:
# extracted rows of which tensors contains 0 (padding).
updated_tensor = tensor[np.any(tensor==0, axis=1)]

In [30]:
print(updated_tensor.shape) 

(150198, 15)


#### 3.3. Input 수정해주기 

In [31]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다. [row, column]
# [:,:-1] all rows and last column
src_input = updated_tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = updated_tensor[:, 1:]  

In [32]:
print(src_input[0])
print(tgt_input[0])


[  2  71 241   5  57 665   3   0   0   0   0   0   0   0]
[ 71 241   5  57 665   3   0   0   0   0   0   0   0   0]


In [33]:
# increased batch size so that steps per epoch is not too big. 
BUFFER_SIZE = len(src_input) #142274
BATCH_SIZE = 1000 
steps_per_epoch = len(src_input) // BATCH_SIZE # 142

# 0:<pad>를 포함하여 +1
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((1000, 14), (1000, 14)), types: (tf.int32, tf.int32)>

### Step 4. 평가 데이터셋 분리

총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!


In [12]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state=7)

In [13]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (120158, 14)
Target Train: (120158, 14)


### Step 5. 인공지능 만들기
* 모델의 Embedding Size와 Hidden Size를 조절
* 10 Epoch 내외
* val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

#### Model 1
same embedding size and hiddensize as LMS node

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

#### Model 2
increased embedding size to 512. hidden size same as Model 1

In [34]:
class TextGenerator2(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size2 = 512
hidden_size2 = 1024
model2 = TextGenerator2(tokenizer.num_words + 1, embedding_size2 , hidden_size2)

#### Model 3
increased embedding size to 1000 and increased hidden size to 1500

In [35]:
class TextGenerator3(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size3 = 1000
hidden_size3 = 1500
model3 = TextGenerator3(tokenizer.num_words + 1, embedding_size3 , hidden_size3)

#### Model 1 fitting 

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)


Epoch 1/10
150/150 [==============================] - 188s 1s/step - loss: 4.8133
Epoch 2/10
150/150 [==============================] - 185s 1s/step - loss: 3.2698
Epoch 3/10
150/150 [==============================] - 187s 1s/step - loss: 3.0669
Epoch 4/10
150/150 [==============================] - 188s 1s/step - loss: 2.9457
Epoch 5/10
150/150 [==============================] - 189s 1s/step - loss: 2.8619
Epoch 6/10
150/150 [==============================] - 189s 1s/step - loss: 2.7949
Epoch 7/10
150/150 [==============================] - 190s 1s/step - loss: 2.7279
Epoch 8/10
150/150 [==============================] - 189s 1s/step - loss: 2.6624
Epoch 9/10
150/150 [==============================] - 189s 1s/step - loss: 2.6109
Epoch 10/10
150/150 [==============================] - 189s 1s/step - loss: 2.5577


#### Model 2 fitting 

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model2.compile(loss=loss, optimizer=optimizer)
model2.fit(dataset, epochs=8)


Epoch 1/8
150/150 [==============================] - 230s 2s/step - loss: 4.9126
Epoch 2/8
150/150 [==============================] - 229s 2s/step - loss: 3.3566
Epoch 3/8
150/150 [==============================] - 231s 2s/step - loss: 3.1455
Epoch 4/8
150/150 [==============================] - 231s 2s/step - loss: 3.0010
Epoch 5/8
150/150 [==============================] - 231s 2s/step - loss: 2.9027
Epoch 6/8
150/150 [==============================] - 231s 2s/step - loss: 2.8305
Epoch 7/8
150/150 [==============================] - 231s 2s/step - loss: 2.7632
Epoch 8/8
150/150 [==============================] - 231s 2s/step - loss: 2.6993


#### Model 3 fitting 

In [36]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model3.compile(loss=loss, optimizer=optimizer)
model3.fit(dataset, epochs=10)

Epoch 1/10
150/150 [==============================] - 511s 3s/step - loss: 4.7749
Epoch 2/10
150/150 [==============================] - 511s 3s/step - loss: 3.3155
Epoch 3/10
150/150 [==============================] - 512s 3s/step - loss: 3.0978
Epoch 4/10
150/150 [==============================] - 512s 3s/step - loss: 2.9490
Epoch 5/10
150/150 [==============================] - 512s 3s/step - loss: 2.8397
Epoch 6/10
150/150 [==============================] - 512s 3s/step - loss: 2.7403
Epoch 7/10
150/150 [==============================] - 512s 3s/step - loss: 2.6592
Epoch 8/10
150/150 [==============================] - 512s 3s/step - loss: 2.5842
Epoch 9/10
150/150 [==============================] - 512s 3s/step - loss: 2.5102
Epoch 10/10
150/150 [==============================] - 512s 3s/step - loss: 2.4379


### Step 6. Model Testing 

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

#### Model 1 Testing

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a <unk> <end> '

#### Model 2 Testing

In [27]:
generate_text(model2, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

#### Model 3 Testing

In [39]:
generate_text(model3, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '